# **Please UPVOTE The notebook**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### import libraries, and load data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

#### **reading the csv file**

In [ ]:
df = pd.read_csv("/kaggle/input/titanic/train.csv", index_col="PassengerId")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv", index_col="PassengerId")
df

### **starting with Explolatory data analysis (EDA)**

In [ ]:
# shape of the training data
df.shape

In [ ]:
# general info about columns of the data in our hand
df.info()

In [ ]:
# information about numerical columns
df.describe()

In [ ]:
# number of null values in each column
df.isna().sum()

In [ ]:
# here the percentage of null values in each column
df.isna().mean()

## columns that will be dropped
**["Name", "Cabin", "Ticket", "Fare"]**
#### 1- name column as names doesn't carry relevant information, the same for (Ticket and Fare)
#### 2- 77% of Cabin column is null values, so i will delete that column as we cann't impute it accurately

In [ ]:
label = "Survived"

In [ ]:
df.columns

In [ ]:
df["Age"].unique(), df["Age"].value_counts()

In [ ]:
df["Pclass"].hist()

In [ ]:
# df.Survived.plot.bar()

In [ ]:
df.boxplot(column="Pclass")

In [ ]:
plt.bar(df["Pclass"].value_counts().index, df["Pclass"].value_counts())
plt.grid()

In [ ]:
print(df["Pclass"].unique())
sns.catplot(x=label, col="Pclass", kind="count", data=df)

In [ ]:
print(df["SibSp"].unique())
sns.catplot(x="SibSp", col=label, kind="count", data=df)
plt.show()

In [ ]:
print(df["Parch"].unique())
sns.catplot(x="Parch", col=label, kind="count", data=df)
plt.show()

In [ ]:
df["Parch"].nunique(), df["Parch"].value_counts()

In [ ]:
sns.catplot(x="Fare", col=label, kind="count", data=df)

In [ ]:
df["Fare"].nunique(), df["Fare"].value_counts()

#### Fare column also will be deleted

In [ ]:
sns.catplot(x="Ticket", col=label, kind='count', data=df);

In [ ]:
df["Ticket"].nunique(), df["Ticket"].value_counts()

#### also Ticket column needed to be deleted

### **using train test split function to split the training data into training and validation set**

In [ ]:
y = df[label]
x = df.drop(label, axis=1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=7, stratify=y)

### *we will do all the preprocessing steps using the make_column_transformer this has many advantages, such as we can involve the preprocessing step in the fine tuning process, and we dont need the repeat the same steps for testing data step by step*

In [ ]:
df.columns

In [ ]:
x = df.drop(label, axis=1)
col_trans = make_column_transformer(("drop", ["Name", "Cabin", "Ticket", "Fare"]),
                                    (KNNImputer(), ["Age"]), 
                                    (SimpleImputer(strategy="most_frequent"), ['Embarked']), 
                                    (OneHotEncoder(), ['Sex']),
#                                     (OneHotEncoder(drop="first"), ['Pclass', 'SibSp', 'Parch']),
                                    remainder="passthrough")
col_trans_1 = make_column_transformer( 
                                    (StandardScaler(), [0]),    #Age
                                    (OneHotEncoder(), [1]),    #Embarked
                                     remainder="passthrough")
vt = VarianceThreshold(threshold=0.01)

# col_trans.fit_transform(xtrain)

In [ ]:


from sklearn.svm import SVC
clf = SVC()
pipe = make_pipeline(col_trans,
                     col_trans_1,
                     vt,
                    clf)
param_grid={

}
search = GridSearchCV(pipe, param_grid, cv=5)
search.fit(xtrain, ytrain)
search.best_score_



In [ ]:
# clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)
clf = RandomForestClassifier(max_depth=5, n_estimators=10)
pipe = make_pipeline(col_trans, col_trans_1, vt, clf
                    )
param_grid = {}
rf_search = GridSearchCV(pipe, param_grid)
rf_search.fit(xtrain, ytrain)
rf_search.best_score_, confusion_matrix(ytest, rf_search.best_estimator_.predict(xtest))

In [ ]:
search = rf_search
best_model = search.best_estimator_
best_model.fit(x, y)
pred = best_model.predict(test_data)

index = pd.read_csv("/kaggle/input/titanic/test.csv")["PassengerId"]
my_submission = pd.DataFrame({'PassengerId': index, "Survived": pred})
my_submission.to_csv('submission.csv', index=False)